In [3]:
import numpy as np
import pandas as pd
import scipy.stats
import sklearn_crfsuite
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn_crfsuite import metrics

In [10]:
DATA = "HR" 

if DATA == "PTP": 
    
    df_test = pd.read_csv('../../data_v3/ptp_il_tagged.csv')
    
    df_train = pd.read_csv('../../data_v3/ptp_extended_features.csv')
    
    df_test.rename(columns={'real_activity_action': "ActivityAction"}, inplace=True)
    
    df_gt = pd.read_csv('../../data_v3/ptp_ground_truth.csv')
    
elif DATA == "HR": 
    
    df_test = pd.read_csv('../../data_v3/hr_il_tagged.csv')

    df_train = pd.read_csv('../../data_v3/hr_extended_features.csv')
    df_test.drop(columns=['ActivityAction'], inplace=True)
    df_test.rename(columns={'real_activity_action': "ActivityAction"}, inplace=True)
    
    df_gt = pd.read_csv('../../data_v3/hr_ground_truth.csv')

In [11]:
def dict_to_features(dict):
    return [[{**d, "bias": 1.0}] for d in dict]


def extract_labels(labels):
    return [[y] for y in labels]

In [12]:
cols = ["event_with_roles", "request_method_call", "selective_filter_data","origin_request_method","origin_selective_filter_data"]
#cols = ["event_with_roles", "request_method_call", "selective_filter_data","origin_selective_filter_data"]

train_features = df_train[cols].to_dict("records")
train_features = dict_to_features(train_features)
train_labels = extract_labels(df_train["ActivityAction"])

In [13]:
test_features = df_test[cols].to_dict("records")
test_features = dict_to_features(test_features)
test_labels = extract_labels(df_test["ActivityAction"])

In [14]:
df_test[cols]["request_method_call"].value_counts()

request_method_call
execute_kw        135
id                 67
version            37
authenticate       37
server_version     37
6                  34
1                  33
faultCode          10
7                   3
739                 1
1231                1
8007                1
8006                1
1233                1
737                 1
736                 1
7996                1
8018                1
1232                1
7992                1
738                 1
1230                1
735                 1
734                 1
7979                1
733                 1
1229                1
1228                1
7965                1
1227                1
732                 1
1226                1
1225                1
1224                1
8023                1
Name: count, dtype: int64

In [15]:
df_train[cols]["request_method_call"].value_counts()

request_method_call
execute_kw        3479
id                1714
server_version     932
authenticate       932
version            927
                  ... 
905                  1
906                  1
907                  1
908                  1
1078                 1
Name: count, Length: 667, dtype: int64

In [16]:
%%time
import sklearn_crfsuite

crf = sklearn_crfsuite.CRF(
    max_iterations=200,
    c1=0.1,
    c2=0.01,
    all_possible_transitions=True
    #all_possible_transitions=True
)
crf.fit(train_features, train_labels)

CPU times: user 975 ms, sys: 17 ms, total: 992 ms
Wall time: 1.41 s


CRF(all_possible_transitions=True, c1=0.1, c2=0.01, max_iterations=200)

In [17]:
from sklearn.metrics import multilabel_confusion_matrix
from sklearn_crfsuite import metrics


def flatten(xss):
    return [x for xs in xss for x in xs]


def evaluate(model, x, y_true):
    y_pred = model.predict(x)
    print(metrics.flat_f1_score(y_true, y_pred, average='macro', labels=model.classes_))
    print(metrics.flat_classification_report(y_true, y_pred, model.classes_))
    [print(label, "\n", matrix) for matrix, label in
     zip(multilabel_confusion_matrix(flatten(y_true), flatten(y_pred), labels=model.classes_), model.classes_)]


evaluate(crf, test_features, test_labels)

0.9608172143245554
                precision    recall  f1-score   support

Activity Start       1.00      1.00      1.00        37
      NoAction       1.00      1.00      1.00      1224
  Activity End       0.94      0.84      0.89        37

      accuracy                           0.99      1298
     macro avg       0.98      0.95      0.96      1298
  weighted avg       0.99      0.99      0.99      1298

Activity Start 
 [[1261    0]
 [   0   37]]
NoAction 
 [[  68    6]
 [   2 1222]]
Activity End 
 [[1259    2]
 [   6   31]]


In [ ]:
from sklearn.metrics import make_scorer
import scipy
from sklearn.model_selection import RandomizedSearchCV
from sklearn_crfsuite import metrics

# define fixed parameters and parameters to search
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    max_iterations=200,
    all_possible_transitions=True
)
params_space = {
    'c1': scipy.stats.expon(scale=0.5),
    'c2': scipy.stats.expon(scale=0.05),
}

# use the same metric for evaluation
f1_scorer = make_scorer(metrics.flat_f1_score,
                        average='macro', labels=np.unique(test_labels))

# search
rs = RandomizedSearchCV(crf, params_space,
                        cv=5,
                        verbose=1,
                        n_jobs=-1,
                        n_iter=150,
                        scoring=f1_scorer)
rs.fit(train_features, train_labels)

crf = rs.best_estimator_

Fitting 5 folds for each of 150 candidates, totalling 750 fits
